In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importing dataset

In [ ]:
dataset=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
dataset

In [ ]:
dataset.info()

In [ ]:
dataset['sentiment'].value_counts()

# Explanotary Data Analysis

In [ ]:
dataset.groupby('sentiment').size().plot(kind='bar')

In [ ]:
pos_sent=dataset[dataset['sentiment']=='positive']
pos_sent

In [ ]:
from wordcloud import WordCloud,STOPWORDS

In [ ]:
pos_words=' '.join(pos_sent['review'])

pos_cloud=WordCloud(width=1000,height=500).generate(pos_words)
plt.figure(figsize=(16,10))
plt.imshow(pos_cloud)
plt.axis('off')
plt.show()

In [ ]:
neg_sent=dataset[dataset['sentiment']=='negative']
neg_sent

In [ ]:
neg_words=' '.join(neg_sent['review'])

neg_cloud=WordCloud(width=1000,height=500).generate(neg_words)
plt.figure(figsize=(16,10))
plt.imshow(neg_cloud)
plt.axis('off')
plt.show()

In [ ]:
dataset['sentiment']=dataset['sentiment'].replace(to_replace='positive',value=1)
dataset['sentiment']=dataset['sentiment'].replace(to_replace='negative',value=0)

In [ ]:
dataset

# Creating a corpus

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
ps=WordNetLemmatizer()
corpus=[]

for i in range(0,len(dataset)):
    reviews=re.sub('[^a-zA-Z]',' ',dataset['review'][i])
    reviews=reviews.lower()
    reviews=reviews.split()
    reviews=[ps.lemmatize(word) for word in reviews if not word in set(stopwords.words('english'))]
    reviews=' '.join(reviews)
    corpus.append(reviews)

# Creating Bag of words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2000)
x=cv.fit_transform(corpus).toarray()

In [ ]:
y=dataset.iloc[:,[1]].values

In [ ]:
y

# Splitting the dataset

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=1/4,random_state=0)

# Training model

1. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train,y_train)
lr_pred=lr.predict(x_test)

In [ ]:
y_test

In [ ]:
lr_pred

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm_lr=confusion_matrix(y_test,lr_pred)
cm_lr

In [ ]:
acc_lr=accuracy_score(y_test,lr_pred)
acc_lr

2. Gaussian Naive Bayes Classification 

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()
gnb.fit(x_train,y_train)
gnb_pred=gnb.predict(x_test)

In [ ]:
cm_gnb=confusion_matrix(y_test,gnb_pred)
cm_gnb

In [ ]:
acc_gnb=accuracy_score(y_test,gnb_pred)
acc_gnb

3. Decision Tree Classifier 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(random_state=0)
dtc.fit(x_train,y_train)
dtc_pred=dtc.predict(x_test)

In [ ]:
cm_dtc=confusion_matrix(y_test,dtc_pred)
cm_dtc

In [ ]:
acc_dtc=accuracy_score(y_test,dtc_pred)
acc_dtc

4. Random Forest Classification 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=400,random_state=0)
rfc.fit(x_train,y_train)
rfc_pred=rfc.predict(x_test)

In [ ]:
cm_rfc=confusion_matrix(y_test,rfc_pred)
cm_rfc

In [ ]:
acc_rfc=accuracy_score(y_test,rfc_pred)
acc_rfc

# Results

In [ ]:
print("Logistic Regression"+" Accuracy={:.2%}".format(acc_lr))
print("Guassian Naive Bayes"+" Accuracy={:.2%}".format(acc_gnb))
print("Decision Tree Classifier"+" Accuracy={:.2%}".format(acc_dtc))
print("Random Forest Classifier"+" Accuracy={:.2%}".format(acc_rfc))

# Vader Analysis for given Dataset

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [ ]:
analyser=SentimentIntensityAnalyzer()

In [ ]:
def sentiment_chk(sentence):
    k=analyser.polarity_scores(sentence)
    return k

In [ ]:
str1="Wonder Woman 2 was a really good movie."
sentiment_chk(str1)

In [ ]:
dataset['scores']=dataset['review'].apply(lambda x: sentiment_chk(x))

In [ ]:
dataset

In [ ]:
dataset.sample(20)